In [1]:
# settingup modules
import pandas as pd
import numpy as np

In [6]:
# importing data
subscription_tracts = pd.read_csv("ACSDT5Y2018.B28002_data_with_overlays_2020-04-08T122811.csv", skiprows=[0])

In [7]:
# some data cleaning
## splitting Geographic Area Name
subscription_tracts = subscription_tracts.join(pd.DataFrame(subscription_tracts["Geographic Area Name"].str.split(",").tolist(),
                                                           columns = ["tract", "county", "city"]))
## Replacing County Name
subscription_tracts["county"].replace({" Bronx County": "Bronx",
                                       " Queens County": "Queens",
                                       " Kings County": "Brooklyn",
                                       " Richmond County": "Staten Island",
                                       " New York County": "Manhattan"}, 
                                      inplace = True)
## Subscripting for Census Tract Number
subscription_tracts["tract"] = subscription_tracts["tract"].apply(lambda x: x[13:])
## Adding Boro Census Tract Label for NTA Use later
subscription_tracts["BoroCTLbl"] = subscription_tracts["county"] + " " + subscription_tracts["tract"].astype(str)

In [8]:
## Renaming desired original Census Bureau columns
renamed_cols = {
    'Estimate!!Total': 'Total',
    'Estimate!!Total!!With an Internet subscription!!Cellular data plan!!Cellular data plan with no other type of Internet subscription' : 'Mobile_Dependent',
     'Estimate!!Total!!With an Internet subscription!!Broadband such as cable, fiber optic or DSL': 'Wired_Broadband',
     'Estimate!!Total!!No Internet access': 'No_Internet'
}
## ID desired columns
cols_to_keep = ['city', 'tract', 'county', "BoroCTLbl", 'Total', 
                'Mobile_Dependent', 'Wired_Broadband', 'No_Internet']
## Rename columns
subscription_tracts.rename(columns=renamed_cols, inplace=True)
## Select columns and create new df
subscription_tracts_export = subscription_tracts[cols_to_keep]

In [9]:
# Export as CSV
subscription_tracts_export.to_csv("subscription_acs_2018.csv", index = False)